### Used Car Prediction Problem

1) Problem Statement:
    - This dataset comprises used cars sold on cardekho.com in India as well as important features of these cars.
    - If user can predict the price of the car based on input features.
    - Prediction results can be used to give new seller the price suggestion based on market condition.

2) Data Collection:
    - The dataset is collected from scrapping from cardekho website
    - The data consists of 13 columns and 15411 rows.

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

from MachineLearning.Practical.RandomForestImplementation import model_params

warnings.filterwarnings('ignore')

%matplotlib inline

In [38]:
df = pd.read_csv(r'Datasets/cardekho_imputated.csv', index_col=[0])

In [39]:
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


## Data Cleaning
1) Handling Missing Values
2) Handling Duplicates
3) Check Data Type
4) Understand the Dataset

In [40]:
#Check null values
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [41]:
#Remove unnecessary columns
df.drop('car_name', axis=1, inplace=True)
df.drop('brand', axis=1, inplace=True)

In [42]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [43]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [44]:
#Getting all different types of features
cat_features = [features for features in df.columns if df[features].dtype == 'O']
print(f"Number of Categorical features: {len(cat_features)}")
num_features = [features for features in df.columns if df[features].dtype != 'O']
print(f"Number of Numerical features: {len(num_features)}")
discrete_features = [features for features in num_features if len(df[features].unique()) <= 25]
print(f"Number of Discrete features: {len(discrete_features)}")
continous_features = [features for features in num_features if features not in discrete_features]
print(f"Number of Continous features: {len(continous_features)}")

Number of Categorical features: 4
Number of Numerical features: 7
Number of Discrete features: 2
Number of Continous features: 5


In [45]:
from sklearn.model_selection import train_test_split
X=df.drop(['selling_price'], axis=1)
y=df['selling_price']

In [46]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


## Feature Engineering and Scaling
### One Hot Encoding for Columns which had lesser unique values and not ordinal
   - One hot Encoding is a process by which categorical variables are converted into a form that could be provided to ML algorithms to do a better job in prediction

In [47]:
len(df['model'].unique())

120

In [48]:
df['model'].value_counts()

model
i20             906
Swift Dzire     890
Swift           781
Alto            778
City            757
               ... 
Altroz            1
C                 1
Ghost             1
Quattroporte      1
Gurkha            1
Name: count, Length: 120, dtype: int64

In [49]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X['model']=le.fit_transform(X['model'])

In [50]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [51]:
X.columns

Index(['model', 'vehicle_age', 'km_driven', 'seller_type', 'fuel_type',
       'transmission_type', 'mileage', 'engine', 'max_power', 'seats'],
      dtype='object')

In [52]:
len(df['seller_type'].unique())

3

In [53]:
df['fuel_type'].unique()

array(['Petrol', 'Diesel', 'CNG', 'LPG', 'Electric'], dtype=object)

In [54]:
df['transmission_type'].unique()

array(['Manual', 'Automatic'], dtype=object)

In [55]:
## Create column transformer with 3 types of transformers
num_features = X.select_dtypes(exclude='object').columns
onehot_columns=[ 'seller_type', 'fuel_type', 'transmission_type']
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_transformer=StandardScaler()
oh_tranformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer([
    ("OneHotEncoder", oh_tranformer, onehot_columns),
    ("StandardScaler", numeric_transformer, num_features)
], remainder='passthrough')

In [56]:
X  = preprocessor.fit_transform(X)

In [57]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,1.247335,-0.000276,-1.324259,-1.263352,-0.403022
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.225693,-0.343933,-0.690016,-0.192071,-0.554718,-0.432571,-0.403022
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.536377,1.647309,0.084924,-0.647583,-0.554718,-0.479113,-0.403022
3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,-0.360667,0.292211,-0.936610,-0.779312,-0.403022
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.666211,-0.012060,-0.496281,0.735736,0.022918,-0.046502,-0.403022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15406,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.508844,0.983562,-0.869744,0.026096,-0.767733,-0.757204,-0.403022
15407,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.556082,-1.339555,-0.728763,-0.527711,-0.216964,-0.220803,2.073444
15408,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.407551,-0.012060,0.220539,0.344954,0.022918,0.068225,-0.403022
15409,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.426247,-0.343933,72.541850,-0.887326,1.329794,0.917158,2.073444


In [58]:
## Separate the dataset into train and tests splits
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((12328, 14), (3083, 14))

In [59]:
X_train

array([[ 0.        ,  0.        ,  1.        , ...,  1.75390551,
         2.66249771, -0.40302241],
       [ 1.        ,  0.        ,  0.        , ..., -0.55087963,
        -0.38602844, -0.40302241],
       [ 0.        ,  0.        ,  1.        , ...,  0.89033072,
         3.27453006, -0.40302241],
       ...,
       [ 1.        ,  0.        ,  0.        , ..., -0.9366097 ,
        -0.78070786, -0.40302241],
       [ 0.        ,  0.        ,  0.        , ..., -0.55471774,
        -0.43582879, -0.40302241],
       [ 1.        ,  0.        ,  0.        , ..., -0.04616815,
         0.06194201, -0.40302241]], shape=(12328, 14))

## Model Training and Model Selection

In [60]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


In [61]:
## Create a function to evaluate model
def evaluate_model(true, prediction):
    mae = mean_absolute_error(true, prediction)
    mse = mean_squared_error(true, prediction)
    rmse = np.sqrt(mse)
    r2_square = r2_score(true, prediction)
    return mae,mse,rmse, r2_square

In [62]:
#Beginning model training
models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "K-nearest Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor()
}


In [64]:
for key, value in models.items():
    model = value
    model.fit(X_train, y_train)

    ##Make predictions
    y_train_predict = model.predict(X_train)
    y_test_predict = model.predict(X_test)

    #Evaluate Train Test Model
    model_train_mae, model_train_mse , model_train_rmse,  model_train_r2_sqaure = evaluate_model(y_train, y_train_predict)
    model_test_mae, model_test_mse , model_test_rmse,  model_test_r2_sqaure = evaluate_model(y_test, y_test_predict)


    print(f'Model : {key}')

    print("\n===== TRAINING METRICS =====")
    print(f"MAE: {model_train_mae:.4f}")
    print(f"MSE: {model_train_mse:.4f}")
    print(f"RMSE: {model_train_rmse:.4f}")
    print(f"R² Score: {model_train_r2_sqaure:.4f}")

    print("\n===== TEST METRICS =====")
    print(f"MAE: {model_test_mae:.4f}")
    print(f"MSE: {model_test_mse:.4f}")
    print(f"RMSE: {model_test_rmse:.4f}")
    print(f"R² Score: {model_test_r2_sqaure:.4f}")
    print("==" *30)

Model : Linear Regression

===== TRAINING METRICS =====
MAE: 268101.6071
MSE: 306756099359.7596
RMSE: 553855.6665
R² Score: 0.6218

===== TEST METRICS =====
MAE: 279618.5794
MSE: 252550062888.5655
RMSE: 502543.5930
R² Score: 0.6645
Model : Ridge

===== TRAINING METRICS =====
MAE: 268059.8015
MSE: 306756818740.9266
RMSE: 553856.3160
R² Score: 0.6218

===== TEST METRICS =====
MAE: 279557.2169
MSE: 252540243247.9685
RMSE: 502533.8230
R² Score: 0.6645
Model : Lasso

===== TRAINING METRICS =====
MAE: 268099.2226
MSE: 306756104248.3742
RMSE: 553855.6710
R² Score: 0.6218

===== TEST METRICS =====
MAE: 279614.7461
MSE: 252549134806.7813
RMSE: 502542.6696
R² Score: 0.6645
Model : K-nearest Neighbors Regressor

===== TRAINING METRICS =====
MAE: 91425.6327
MSE: 106193229526.2816
RMSE: 325873.0267
R² Score: 0.8691

===== TEST METRICS =====
MAE: 112526.3461
MSE: 64021344520.1508
RMSE: 253024.3951
R² Score: 0.9150
Model : Decision Tree

===== TRAINING METRICS =====
MAE: 5164.8199
MSE: 432524990.5364

In [65]:
## The models which performed well are: KNN and RF
## Hyperparameter tuning for these models
knn_params={"n_neighbors":[2,3,10,20,40,50]}
rf_params = {
    'max_depth':[5,8,15,None,10],
    'max_features':[5,7,'auto',8],
    'min_samples_split':[2,8,15,20],
    'n_estimators':[100,200,500,1000]
}

In [66]:
## Model List for hyperparameter tuning
randomcv_models = [("KNN",KNeighborsRegressor(), knn_params),
                   ("RF", RandomForestRegressor(), rf_params)]

In [67]:
#Hyperparameter Tuning
from sklearn.model_selection import RandomizedSearchCV
model_params={}
for name, model, param in randomcv_models:
    random = RandomizedSearchCV(estimator=model,
                                param_distributions=param,
                                n_iter=100,
                                cv=3,
                                verbose=2,
                                n_jobs=-1)
    random.fit(X_train, y_train)
    model_params[name] = random.best_params_

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [68]:
for model_name in model_params:
    print(f'------ Best params for {model_name} --------')
    print(model_params[model_name])

------ Best params for KNN --------
{'n_neighbors': 10}
------ Best params for RF --------
{'n_estimators': 500, 'min_samples_split': 2, 'max_features': 8, 'max_depth': None}


In [69]:
## Train the model again with best params
models = {
    "K-nearest Neighbors Regressor": KNeighborsRegressor(n_neighbors= 10),
    "Random Forest Regressor": RandomForestRegressor(n_estimators= 500, min_samples_split= 2, max_features= 8, max_depth= None)
}

In [70]:
for key, value in models.items():
    model = value
    model.fit(X_train, y_train)

    ##Make predictions
    y_train_predict = model.predict(X_train)
    y_test_predict = model.predict(X_test)

    #Evaluate Train Test Model
    model_train_mae, model_train_mse , model_train_rmse,  model_train_r2_sqaure = evaluate_model(y_train, y_train_predict)
    model_test_mae, model_test_mse , model_test_rmse,  model_test_r2_sqaure = evaluate_model(y_test, y_test_predict)


    print(f'Model : {key}')

    print("\n===== TRAINING METRICS =====")
    print(f"MAE: {model_train_mae:.4f}")
    print(f"MSE: {model_train_mse:.4f}")
    print(f"RMSE: {model_train_rmse:.4f}")
    print(f"R² Score: {model_train_r2_sqaure:.4f}")

    print("\n===== TEST METRICS =====")
    print(f"MAE: {model_test_mae:.4f}")
    print(f"MSE: {model_test_mse:.4f}")
    print(f"RMSE: {model_test_rmse:.4f}")
    print(f"R² Score: {model_test_r2_sqaure:.4f}")
    print("==" *30)

Model : K-nearest Neighbors Regressor

===== TRAINING METRICS =====
MAE: 103471.6175
MSE: 132103680886.5996
RMSE: 363460.7006
R² Score: 0.8371

===== TEST METRICS =====
MAE: 117487.1310
MSE: 69633846174.3837
RMSE: 263882.2582
R² Score: 0.9075
Model : Random Forest Regressor

===== TRAINING METRICS =====
MAE: 38999.5099
MSE: 16194857301.8715
RMSE: 127259.0166
R² Score: 0.9800

===== TEST METRICS =====
MAE: 98597.7693
MSE: 45735923971.7203
RMSE: 213859.5894
R² Score: 0.9392
